In [6]:
import pandas as pd
import yfinance as yf
import investpy
import datetime as dt
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

In [20]:
start_date = dt.date.today() - dt.timedelta(days=365*10)
end_date = dt.date.today()



# Fetching Niftybees data
niftybees = yf.download(['NIFTYBEES.NS','INR=X', '^GSPC', 'CL=F'], start=start_date, end=end_date,interval='1mo')['Adj Close']



[*********************100%%**********************]  4 of 4 completed


In [34]:
# Fetching other exogenous variables

cpi_data_0 = investpy.economic_calendar(countries=['India'], from_date=pd.to_datetime(start_date, format='%Y-%m-%d').strftime('%d/%m/%Y'), to_date=pd.to_datetime(end_date, format='%Y-%m-%d').strftime('%d/%m/%Y'))
# Filter rows where 'event' contains 'Consumer Price Index' or related terms
keywords = ['Consumer Price Index', 'CPI']

# Create a filter condition
condition = cpi_data_0['event'].str.contains('|'.join(keywords), case=False, na=False)

# Apply the filter
cpi_data = cpi_data_0[condition]


In [24]:
niftybees

Ticker,CL=F,INR=X,NIFTYBEES.NS,^GSPC
Date,,,,
2014-09-01,91.160004,61.553001,80.718002,1972.290039
2014-10-01,80.540001,61.432999,84.114998,2018.050049
2014-11-01,66.150002,61.853001,86.931000,2067.560059
2014-12-01,53.270000,63.377998,83.876999,2058.899902
2015-01-01,48.240002,62.013000,89.482002,1994.989990
...,...,...,...,...
2024-04-01,81.930000,83.451103,250.059998,5035.689941
2024-05-01,76.989998,83.320900,251.149994,5277.509766
2024-06-01,81.540001,83.450897,267.480011,5460.479980


In [35]:
cpi_data = cpi_data[["date","event","actual"]]
cpi_data["date"] = pd.to_datetime(pd.to_datetime(cpi_data["date"], format='%d/%m/%Y').dt.strftime('%Y-%m-01'))
cpi_data

,date,event,actual
11,2014-09-01,CPI (YoY) (Aug),7.80%
46,2014-10-01,CPI (YoY) (Aug),6.46%
73,2014-11-01,CPI (YoY) (Oct),5.52%
105,2014-12-01,CPI (YoY) (Nov),4.38%
131,2015-01-01,CPI (YoY) (Dec),5.00%
...,...,...,...
5248,2024-04-01,CPI (YoY) (Mar),4.85%
5280,2024-05-01,CPI (YoY) (Apr),4.83%
5313,2024-06-01,CPI (YoY) (May),4.75%
5356,2024-07-01,CPI (YoY) (Jun),5.08%


In [36]:
cpi_data = cpi_data.set_index(["date"])
cpi_data

,event,actual
date,,
2014-09-01,CPI (YoY) (Aug),7.80%
2014-10-01,CPI (YoY) (Aug),6.46%
2014-11-01,CPI (YoY) (Oct),5.52%
2014-12-01,CPI (YoY) (Nov),4.38%
2015-01-01,CPI (YoY) (Dec),5.00%
...,...,...
2024-04-01,CPI (YoY) (Mar),4.85%
2024-05-01,CPI (YoY) (Apr),4.83%
2024-06-01,CPI (YoY) (May),4.75%


In [38]:
niftybees_df = pd.concat([niftybees, cpi_data],axis=1)
niftybees_df

,CL=F,INR=X,NIFTYBEES.NS,^GSPC,event,actual
2014-09-01,91.160004,61.553001,80.718002,1972.290039,CPI (YoY) (Aug),7.80%
2014-10-01,80.540001,61.432999,84.114998,2018.050049,CPI (YoY) (Aug),6.46%
2014-11-01,66.150002,61.853001,86.931000,2067.560059,CPI (YoY) (Oct),5.52%
2014-12-01,53.270000,63.377998,83.876999,2058.899902,CPI (YoY) (Nov),4.38%
2015-01-01,48.240002,62.013000,89.482002,1994.989990,CPI (YoY) (Dec),5.00%
...,...,...,...,...,...,...
2024-04-01,81.930000,83.451103,250.059998,5035.689941,CPI (YoY) (Mar),4.85%
2024-05-01,76.989998,83.320900,251.149994,5277.509766,CPI (YoY) (Apr),4.83%
2024-06-01,81.540001,83.450897,267.480011,5460.479980,CPI (YoY) (May),4.75%
2024-07-01,77.910004,83.738998,277.750000,5522.299805,CPI (YoY) (Jun),5.08%


In [40]:
niftybees_df.columns = ['Crude_oil', 'Exchange_rate', 'NIFTYBEES', 'S&P_500', 'CPI_event', 'CPI_actual_growth']
niftybees_df

,Crude_oil,Exchange_rate,NIFTYBEES,S&P_500,CPI_event,CPI_actual_growth
2014-09-01,91.160004,61.553001,80.718002,1972.290039,CPI (YoY) (Aug),7.80%
2014-10-01,80.540001,61.432999,84.114998,2018.050049,CPI (YoY) (Aug),6.46%
2014-11-01,66.150002,61.853001,86.931000,2067.560059,CPI (YoY) (Oct),5.52%
2014-12-01,53.270000,63.377998,83.876999,2058.899902,CPI (YoY) (Nov),4.38%
2015-01-01,48.240002,62.013000,89.482002,1994.989990,CPI (YoY) (Dec),5.00%
...,...,...,...,...,...,...
2024-04-01,81.930000,83.451103,250.059998,5035.689941,CPI (YoY) (Mar),4.85%
2024-05-01,76.989998,83.320900,251.149994,5277.509766,CPI (YoY) (Apr),4.83%
2024-06-01,81.540001,83.450897,267.480011,5460.479980,CPI (YoY) (May),4.75%
2024-07-01,77.910004,83.738998,277.750000,5522.299805,CPI (YoY) (Jun),5.08%
